# 베드락을 이용해서 상품 이미지에서 속성 추출/분류하기
> *이 노트북은 SageMaker Studio*의 `JupyterLab` 에서 테스트하였습니다. 

## 소개

이 노트북에서는 리테일 고객이 상품 이미지에서 속성을 추출하고 분류하는 방법을 보여드리겠습니다. 상품 이미지로 작업할 때, 이미지의 세부 사항에 대한 정보를 추출하거나, 회사의 기준에 맞게 분류할 수 있습니다. 

이미지 이해 패턴에는 두 가지 입력이 필요합니다:
* 분석할 하나 이상의 이미지
* 이미지에 대해 알고 싶은 내용을 설명하는 프롬프트

Anthropic Claude의 [비전 기능](https://docs.anthropic.com/claude/docs/use-cases-and-capabilities#vision-capabilities)을 사용하여 이미지 이해 예제를 구축하겠습니다.

#### 컨텍스트

이 노트북에서는 LangChain 프레임워크 내에서 Amazon Bedrock과 통합하여 사용는 방법과 PromptTemplate의 도움으로 텍스트를 생성하는 데 어떻게 사용될 수 있는지 살펴보겠습니다.


#### 사용사례

이미지 이해 패턴은 다음과 같은 사용 사례에 적합합니다:

* 접근 가능한 대체 텍스트 및 이미지 캡션 생성
* 이미지 처리 파이프라인에서 이미지가 부적절하거나 관련 콘텐츠와 일치하지 않는지 감지하는 데 사용
* 이미지 처리 파이프라인에서 마스크 프롬프트 및 인페인팅/아웃페인팅 작업을 자동화하는 데 사용
* 이미지 분류
* 이미지에서 텍스트 추출
* [기타](https://docs.anthropic.com/claude/docs/use-cases-and-capabilities#vision-capabilities) 


#### 구현 방법

이 사용 사례를 보여주기 위해 이 노트북에서는 제품의 이미지를 분류하고, 세부 정보를 추출하는 방법을 보여드리며, Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하는 Anthropic Claude 모델을 사용하겠습니다.


## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) 베드락에 연결해야 합니다.

우선 사전에 설치가 필요한 패키지들을 설치하세요. 그 이후에 셋업에 필요한 라이브러리들을 설치합니다. 

#### 앞 부분은 이전 실습 과정에서 했던 내용과 동일합니다

In [ ]:
!pip install -r dependencies/requirements.txt -q

In [ ]:
import json
import os
import sys
import boto3
from botocore.config import Config
import base64
from io import BytesIO

# this is setting the maximum number of times boto3 will attempt our call to bedrock
my_region = "us-west-2" # change this value to point to a different region
my_config = Config(
    region_name = my_region,
    signature_version = 'v4',
    retries = {
        'max_attempts': 3,
        'mode': 'standard'
    }
)

# this creates our client we will use to access Bedrock
bedrock_rt = boto3.client("bedrock-runtime", config = my_config)
bedrock = boto3.client("bedrock")

## 상품 이미지 확인하기

Boto3 라이브러리를 사용하여 Bedrock을 호출하고 이미지 데이터를 처리할 수 있습니다. 파일, 이미지, Base64 인코딩 바이트 간의 데이터 변환도 진행해 봅니다. 

In [ ]:
import boto3
import json
import base64
from io import BytesIO


def get_bytesio_from_bytes(image_bytes):
    image_io = BytesIO(image_bytes)
    return image_io


#파일 바이트에서 base64로 인코딩된 문자열 가져오기
def get_base64_from_bytes(image_bytes):
    resized_io = get_bytesio_from_bytes(image_bytes)
    img_str = base64.b64encode(resized_io.getvalue()).decode("utf-8")
    return img_str


#디스크의 파일에서 바이트 로드하기
def get_bytes_from_file(file_path):
    with open(file_path, "rb") as image_file:
        file_bytes = image_file.read()
    return file_bytes

## 베드락 LLM 모델 호출하기

LLM에서 Bedrock 클래스의 인스턴스를 생성하는 것으로 시작하겠습니다. 여기에는 Amazon Bedrock에서 사용할 수 있는 모델의 ARN인 model_id가 필요합니다.

선택적으로 이전에 생성한 boto3 클라이언트를 전달할 수 있으며, `temperature`, `top_p`, `max_tokens` 또는 `stop_sequences`와 같은 매개 변수를 보유할 수 있는 일부 `model_kwargs`도 전달할 수 있습니다(매개 변수에 대한 자세한 내용은 Amazon Bedrock 콘솔에서 살펴볼 수 있습니다).

Amazon Bedrock에서 사용 가능한 텍스트 생성 모델 ID에 대한 [설명서](https://docs.aws.amazon.com/ko_kr/bedrock/latest/userguide/model-ids-arns.html)를 확인하세요.

모델마다 지원하는 `model_kwargs`가 다르다는 점에 유의하세요.

## 상품 이미지

![상품 이미지](./images/newbalance_hoodie.png)

In [ ]:
#InvokeModel API 호출에 대한 문자열화된 요청 본문 가져오기
def get_image_understanding_request_body(prompt, image_bytes=None, mask_prompt=None, negative_prompt=None):
    input_image_base64 = get_base64_from_bytes(image_bytes)    

modelId="anthropic.claude-3-sonnet-20240229-v1:0"

image_path = 'images/newbalance_hoodie.png'

prompt = """
Please categorize this image into one of the following categories: 
Sneakers, Cap, Accesaries, Jacket, Dress, Hoodie, Shirt. 

Only return the category name in Korean.
"""

with open(image_path, 'rb') as image_file:
    input_image_base64=base64.b64encode(image_file.read()).decode('utf-8')

body = json.dumps(
        { 
        "anthropic_version":"bedrock-2023-05-31",
        "max_tokens": 2000,
        "temperature": 0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg", 
                            "data": input_image_base64,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ]
    }    
)

response = bedrock_rt.invoke_model(body=body, modelId=modelId)

response_body = json.loads(response.get('body').read())
output = response_body['content'][0]['text']

print(output)

![상품 이미지2](./images/cat_tee.png)

In [ ]:
#InvokeModel API 호출에 대한 문자열화된 요청 본문 가져오기
def get_image_understanding_request_body(prompt, image_bytes=None, mask_prompt=None, negative_prompt=None):
    input_image_base64 = get_base64_from_bytes(image_bytes)    

modelId="anthropic.claude-3-sonnet-20240229-v1:0"

image_path = 'images/cat_tee.png'

prompt= """
Please create a comma-separated list of the items found in this image. 
Only return the list of items in Korean.
"""

with open(image_path, 'rb') as image_file:
    input_image_base64=base64.b64encode(image_file.read()).decode('utf-8')

body = json.dumps(
        { 
        "anthropic_version":"bedrock-2023-05-31",
        "max_tokens": 2000,
        "temperature": 0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg", #this doesn't seem to matter?
                            "data": input_image_base64,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ]
    }    
)

response = bedrock_rt.invoke_model(body=body, modelId=modelId)

response_body = json.loads(response.get('body').read())
output = response_body['content'][0]['text']

print(output)

#### 앞 부분은 이전 실습 과정에서 했던 내용과 동일합니다

In [ ]:
%%writefile image_understanding_lib.py
import boto3
import json
import base64
from io import BytesIO


#get a BytesIO object from file bytes
def get_bytesio_from_bytes(image_bytes):
    image_io = BytesIO(image_bytes)
    return image_io


#get a base64-encoded string from file bytes
def get_base64_from_bytes(image_bytes):
    resized_io = get_bytesio_from_bytes(image_bytes)
    img_str = base64.b64encode(resized_io.getvalue()).decode("utf-8")
    return img_str


#load the bytes from a file on disk
def get_bytes_from_file(file_path):
    with open(file_path, "rb") as image_file:
        file_bytes = image_file.read()
    return file_bytes


#get the stringified request body for the InvokeModel API call
def get_image_understanding_request_body(prompt, image_bytes=None, mask_prompt=None, negative_prompt=None):
    input_image_base64 = get_base64_from_bytes(image_bytes)
    
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 2000,
        "temperature": 0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg", #this doesn't seem to matter?
                            "data": input_image_base64,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ],
    }
    
    return json.dumps(body)



#generate a response using Anthropic Claude
def get_response_from_model(prompt_content, image_bytes, mask_prompt=None):
    session = boto3.Session()
    
    bedrock = session.client(service_name='bedrock-runtime') #creates a Bedrock client
    
    body = get_image_understanding_request_body(prompt_content, image_bytes, mask_prompt=mask_prompt)
    
    response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0", contentType="application/json", accept="application/json")
    
    response_body = json.loads(response.get('body').read()) # read the response
    
    output = response_body['content'][0]['text']
    
    return output



In [ ]:
%%writefile ../demo-app.py
import streamlit as st
import PromptEngineering.image_understanding_lib as glib

st.set_page_config(layout="wide", page_title="이미지 이해")

st.title("이미지 이해")

col1, col2, col3 = st.columns(3)

prompt_options_dict = {
   # "Image caption": "Please provide a brief caption for this image.",
    #"Detailed description": "Please provide a thoroughly detailed description of this image.",
    "이미지 분류": "Please categorize this image into one of the following categories: Sneakers, Jacket, Dress, Hoodie, Shirt. Only return the category name in Korean.",
    "속성 추출": "Please create a comma-separated list of the items found in this image. Only return the list of items in Korean.",
    #"Subject identification": "Please name the primary object in the image. Only return the name of the object in <object> tags.",
    #"Writing a story": "Please write a fictional short story based on this image.",
    #"Answering questions": "What emotion are the people in this image displaying?",
    #"Transcribing text": "Please transcribe any text found in this image.",
    #"Translating text": "Please translate the text in this image to French.",
    "기타": "",
}

prompt_options = list(prompt_options_dict)

prefix="./PromptEngineering/images/"
image_options_dict = {
    "Jacket" : f"{prefix}blue-jacket.png",
    "Dress" : f"{prefix}dress.png",
    "Hoodie" : f"{prefix}newbalance_hoodie.png",
    "Shirt" : f"{prefix}oxford-shirts.png",
    "Other": f"{prefix}cat_tee.png",
}

image_options = list(image_options_dict)


with col1:
    st.subheader("이미지를 고르세요")
    
    image_selection = st.radio("이미지 종류:", image_options)
    
    if image_selection == 'Other':
        uploaded_file = st.file_uploader("Select an image", type=['png', 'jpg'], label_visibility="collapsed")
    else:
        uploaded_file = None
    
    if uploaded_file and image_selection == 'Other':
        uploaded_image_preview = glib.get_bytesio_from_bytes(uploaded_file.getvalue())
        st.image(uploaded_image_preview)
    else:
        st.image(image_options_dict[image_selection])
    
    
with col2:
    st.subheader("프롬프트 종류")
    
    prompt_selection = st.radio("Prompt example:", prompt_options)
    
    prompt_example = prompt_options_dict[prompt_selection]
    
    prompt_text = st.text_area("Prompt",
        #value=,
        value=prompt_example,
        height=100,
        help="What you want to know about the image.",
        label_visibility="collapsed")
    
    go_button = st.button("Go", type="primary")
    
    
with col3:
    st.subheader("결과")

    if go_button:
        with st.spinner("Processing..."):
            
            if uploaded_file:
                image_bytes = uploaded_file.getvalue()
            else:
                image_bytes = glib.get_bytes_from_file(image_options_dict[image_selection])
            
            response = glib.get_response_from_model(
                prompt_content=prompt_text, 
                image_bytes=image_bytes,
            )
        
        st.write(response)
